In [1]:
import pandas as pd
import numpy as np

In [150]:
mhs = pd.read_excel('data/data95.xlsx', sheet_name='mahasiswa').dropna()

ruang = pd.read_excel('data/data95.xlsx', sheet_name='ruang', header=None, names=['ruang'])
ruang['kk'] = ['ALL','ICM','TELE','TELE','TELE','TELE','SIDE, MCE','SIDE, MCE','SIDE, MCE','SIDE, MCE','ICM, MCE','ICM, MCE','ICM, MCE']

dosen =  pd.read_excel('data/data95.xlsx', sheet_name='dosen', header=None, na_values='None').iloc[:, [0,-1]].dropna()
dosen.columns = ['kode', 'kk']

jadwal = pd.read_excel('data/data95.xlsx', sheet_name='jadwal')


#masking
_nama = [f"mhs_{i}" for i in range(len(mhs))]
_nim = [f"nim_{i}" for i in range(len(mhs))]
_dosen = [f"dosen_{i}" for i in range(len(dosen))]
_judul = [f"judul_{i}" for i in range(len(mhs))]
dosen_map = dict(zip(dosen.kode, _dosen))

mhs['Nama'] = _nama
mhs['NIM'] = _nim
mhs['Judul TA'] = _judul
mhs[['DosPem 1', 'DosPem 2']] = mhs[['DosPem 1', 'DosPem 2']].replace(dosen_map)

dosen['kode'] = _dosen

jadwal['Teachers'] = jadwal['Teachers'].replace(dosen_map)

#column renaming 
mhs.columns = ['nama', 'nim', 'dp1', 'dp2', 'judul_ta']
jadwal.columns = ['dosen', 'day', 'time']


In [151]:
jadwal.time.unique()

array(['06:30', '13:30', '15:30', '09:30'], dtype=object)

constraint : 
we will be schedule the students on these hours : 
- 08:00
- 09:30
- 13:30
- 15:30

Each student are given 90mins for each schedule

In [152]:
jadwal['val'] = 1
# add '08:00' to the schedule and remove '6:30'

In [153]:
_df = pd.DataFrame(
    [['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat'],
    [0,0,0,0,0],
    ['08:00','08:00','08:00','08:00','08:00'],
    ['dosen_0','dosen_0','dosen_0','dosen_0','dosen_0']],
    index=['day', 'val', 'time', 'dosen']
).T
jadwal = jadwal.append(_df)

In [155]:
cond = jadwal['time'] == '06:30'
jadwal = jadwal[~cond]

In [156]:
jadwal_dosen_wide = jadwal.pivot_table(
    index='dosen', 
    columns=['day', 'time'],
    values='val', 
    aggfunc='sum'
).fillna(0)[['Senin', 'Selasa', 'Rabu', 'Kamis','Jumat']]

In [157]:
from natsort import natsorted
idx = natsorted(jadwal_dosen_wide.index.values.tolist())

In [158]:
jadwal_dosen_wide = jadwal_dosen_wide.reindex(idx)
jadwal_dosen_long = jadwal_dosen_wide.stack(level=0).stack().reset_index().rename(columns={0:'val'})

In [159]:
cond =  mhs['dp2'] == '-'

In [163]:
mhs

,nama,nim,dp1,dp2,judul_ta
0,mhs_0,nim_0,dosen_69,dosen_61,judul_0
1,mhs_1,nim_1,dosen_20,-,judul_1
2,mhs_2,nim_2,dosen_19,dosen_2,judul_2
3,mhs_3,nim_3,dosen_44,-,judul_3
4,mhs_4,nim_4,dosen_4,dosen_2,judul_4
...,...,...,...,...,...
119,mhs_119,nim_119,dosen_43,dosen_48,judul_119
120,mhs_120,nim_120,dosen_33,dosen_45,judul_120
121,mhs_121,nim_121,dosen_44,dosen_36,judul_121
122,mhs_122,nim_122,dosen_33,dosen_38,judul_122


In [164]:
mhs.dp2 = mhs.dp2.replace('-','') + (mhs.dp1 * cond)

In [165]:
mhs

,nama,nim,dp1,dp2,judul_ta
0,mhs_0,nim_0,dosen_69,dosen_61,judul_0
1,mhs_1,nim_1,dosen_20,dosen_20,judul_1
2,mhs_2,nim_2,dosen_19,dosen_2,judul_2
3,mhs_3,nim_3,dosen_44,dosen_44,judul_3
4,mhs_4,nim_4,dosen_4,dosen_2,judul_4
...,...,...,...,...,...
119,mhs_119,nim_119,dosen_43,dosen_48,judul_119
120,mhs_120,nim_120,dosen_33,dosen_45,judul_120
121,mhs_121,nim_121,dosen_44,dosen_36,judul_121
122,mhs_122,nim_122,dosen_33,dosen_38,judul_122


In [167]:
jadwal_dosen_wide.size, len(mhs)

(1580, 124)

In [176]:
mhs.to_csv('mahasiswa.csv', index=False)

In [181]:
jadwal_dosen_long.to_csv('jadwal_dosen.csv', index=False)

In [183]:
ruang.to_csv('ruang.csv', index=False)

In [185]:
dosen.to_csv('dosen.csv', index=False)

# Prblem Complexity
Menjadwalkan : 
- m = 1 mahasiswa (kemunginan : 124)
- d = 1 atau 2 dosen untuk mendampingi (kemungkinan : 3)
- s = pada 1 slot tempat x waktu (kemngkinan: 13 * 20)

Complexity : $$ C_{m \times d}^s$$

Order of Complexity : Combinatorial

In [174]:
comb((124 * 3), (13*20))

30917575600039935319436951333953035261042106084305298821020346950246779237134842949133981196509641